<a href="https://colab.research.google.com/github/cuongko/Capstone-Project-Notebook/blob/main/Buy_Suggestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estabalish Enviroment

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# install API required
! pip3 install beautifulsoup4
! pip3 install requests

In [ ]:
# For scrapping web
import requests
from bs4 import BeautifulSoup

In [ ]:
import numpy as np
import pandas as pd
import sys
import math
import os
import datetime

In [ ]:
# keras and some related libraries
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.models import load_model

In [ ]:
# for visualization
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

# Scrapping Zin Trading

In [ ]:
tech_dict = {'ticker': [], 'ClosePrice': [], 'Pe': [], 'Pb': [], 'IncDec': [],
             'Ichimoku': [],	'RSI_value': [], 'RSI_Action': [],	
             'MFI_Value': [],	'MFI_Action': [],	'MACD_Value': [],	'MACD_Action': []}

for page in range(1,31): # zin data includes of 30 pages
  url = 'https://trading.zinbee.com/Stock?CurPage=' + str(page)
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  results = soup.find_all('td', class_ = 'text-center')

  for i in range(0, len(results)):
    if i % 12 == 0:
      tech_dict['ticker'].append(results[i].text.strip())
    elif i% 12 == 1:
      tech_dict['ClosePrice'].append(results[i].text.strip())
    elif i% 12 == 2:
      tech_dict['Pe'].append(results[i].text.strip())
    elif i% 12 == 3:
      tech_dict['Pb'].append(results[i].text.strip())
    elif i% 12 == 4:
      tech_dict['IncDec'].append(results[i].text.strip())
    elif i% 12 == 5:
      tech_dict['Ichimoku'].append(results[i].text.strip())
    elif i% 12 == 6:
      tech_dict['RSI_value'].append(results[i].text.strip())
    elif i% 12 == 7:
      tech_dict['RSI_Action'].append(results[i].text.strip())
    elif i% 12 == 8:
      tech_dict['MFI_Value'].append(results[i].text.strip())
    elif i% 12 == 9:
      tech_dict['MFI_Action'].append(results[i].text.strip())
    elif i% 12 == 10:
      tech_dict['MACD_Value'].append(results[i].text.strip())
    elif i% 12 == 11:
      tech_dict['MACD_Action'].append(results[i].text.strip())

In [ ]:
Zin_df = pd.DataFrame(tech_dict)
Zin_df.head(5)

,ticker,ClosePrice,Pe,Pb,IncDec,Ichimoku,RSI_value,RSI_Action,MFI_Value,MFI_Action,MACD_Value,MACD_Action
0,STB,23.15,15.77,1.46,-0.30 (-1.28%),MUA (giá: 22.12),69.45,MUA,67.71,MUA,1.13,MUA
1,HPG,50.80,12.62,2.88,-0.40 (-0.78%),TRUNG LẬP,73.39,TRUNG LẬP,86.16,TRUNG LẬP,1.40,MUA
2,MBB,31.20,10.52,1.83,-0.45 (-1.42%),MUA (giá: 29.94),66.29,MUA,63.86,MUA,1.09,MUA
3,CTG,42.65,11.76,1.90,-0.55 (-1.27%),MUA (giá: 41.64),64.86,MUA,67.38,MUA,1.29,MUA
4,SSI,36.15,17.39,2.23,-0.35 (-0.96%),MUA (giá: 34.03),67.01,MUA,71.24,MUA,0.86,MUA


# Predicting Price and Buy suggestion

In [ ]:
nd_trans = 5 # number of day transaction
predict_day = 3 # in the day T+3, stock comes to account
train_test_ratio = 0.7 # train set and test set ratio
# train_days = int(len(df) * train_test_ratio) # number of days to use as train set
sc = MinMaxScaler(feature_range = (0, 1))
data_path = '/content/drive/My Drive/Colab Notebooks/Stock historical data/Updated stock/'
model_path = '/content/drive/My Drive/Colab Notebooks/Stock historical data/My models/'
expected_profit = 5 # expected profit in T5

In [ ]:
# this function is not nesesscary in this section
def get_watchlist(data_path = data_path):
  '''
  get watch list from folder contains stock data file
  '''
  
  files = os.listdir(data_path)
  watchlist = []
  
  for file in files:
    if len(file) <= 15:
      watchlist.append(file[8:11])
  
  return watchlist

In [ ]:
def get_existing_model(model_path = model_path):
  '''
  get name of built models 
  '''
  ext_model = []
  for m in os.listdir(model_path):
    ext_model.append(m[:3])
  
  return ext_model

In [ ]:
def get_newdata(ticker, data_path = data_path):
  '''
  get new data from download file
  '''
 
  df = pd.read_csv(data_path + 'HisData_' + ticker + '.csv')
  columns_order = ['Date',	'PriceClose', 'DealVolume']	
  df = df[columns_order]
  df['Date'] = pd.to_datetime(df['Date'])
  df = df.sort_values(by = 'Date', ascending = True)
  df = df.iloc[len(df) - nd_trans:, 0:]
  
  return df

In [ ]:
def predict_newdata(new_data, model, sc = sc):
  '''
  predict price using new data
  '''
  
  X_new = new_data[['PriceClose',	'DealVolume']].astype('float')
  # transform X new
  X_new = sc.fit_transform(X_new)
  
  # reshape X_new
  X_new = np.reshape(X_new, (1, nd_trans, 2))
  prd_price = [[1,0]]
  
  # predict y
  y_predict = model.predict(X_new)
  y_predict = sc.inverse_transform(y_predict.dot(prd_price))
  
  return y_predict[0,0]

In [ ]:
PriceClose = []
PredictT3 = []
Growth = []
watchlist = get_existing_model()

for ticker in watchlist:
  new_data = get_newdata(ticker)
  model = load_model(model_path + ticker + '.h5')
  PriceClose.append(new_data.iloc[len(new_data)-1, 1])
  PredictT3.append(predict_newdata(new_data = new_data, model = model))
  Growth.append((predict_newdata(new_data = new_data, model = model)/new_data.iloc[len(new_data) - 1 ,1] - 1)*100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


# Suggestion

In [ ]:
df = pd.DataFrame(list(zip(watchlist,PriceClose, PredictT3, Growth)), columns = ['Ticker', 'PriceClose', 'PredictT3', 'Growth'])
df.sort_values(by='Growth', ascending = False, inplace = True)
df.sort_values(by='Growth', ascending = False, inplace = True)

In [ ]:
df['ticker'] = df['Ticker']

In [ ]:
df_temp = df[df['Growth'] >= expected_profit].join(Zin_df.set_index('ticker'), on = 'ticker')[['Ticker', 'PriceClose',	'PredictT3',	'Growth', 'Pe',	'Pb',	'IncDec',	'Ichimoku',	'RSI_value',	'RSI_Action',
                                                                                               'MFI_Value',	'MFI_Action',	'MACD_Value',	'MACD_Action']]
df_temp

,Ticker,PriceClose,PredictT3,Growth,Pe,Pb,IncDec,Ichimoku,RSI_value,RSI_Action,MFI_Value,MFI_Action,MACD_Value,MACD_Action
106,PXT,3.63,4.384413,20.782714,-4.03,1.25,-0.17 (-4.68%),MUA (giá: 3.28),68.78,MUA,49.23,BÁN,0.15,MUA
249,YEG,29.35,34.352904,17.045671,-11.24,1.06,+0.00 (+0.00%),BÁN (giá: 35.12),15.45,TRUNG LẬP,7.13,TRUNG LẬP,-2.34,BÁN
400,HQC,4.06,4.515129,11.210082,201.62,0.45,+0.11 (+2.71%),TRUNG LẬP,77.18,TRUNG LẬP,74.61,MUA,0.30,MUA
415,IDV,78.60,84.963037,8.095467,7.70,3.10,-4.60 (-5.85%),MUA (giá: 81.80),43.04,BÁN MẠNH,45.16,BÁN,2.57,MUA
243,VSC,47.60,51.337971,7.852881,10.87,1.21,+1.80 (+3.78%),BÁN (giá: 53.70),29.91,TRUNG LẬP,24.63,BÁN,-2.55,BÁN
114,RIC,23.10,24.774718,7.249864,-19.90,2.05,-1.60 (-6.93%),BÁN (giá: 27.30),44.37,BÁN MẠNH,59.43,MUA,-0.28,BÁN
404,HTP,16.50,17.611652,6.737282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,NTC,226.00,239.927488,6.162605,18.72,8.09,-4.50 (-1.99%),BÁN (giá: 246.00),27.22,TRUNG LẬP,39.75,BÁN,-6.84,BÁN
91,PVC,11.10,11.713198,5.524305,58.35,0.66,-0.30 (-2.70%),MUA (giá: 11.00),50.45,MUA,59.09,MUA,0.35,MUA
221,VIS,18.55,19.552975,5.406873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# saving predict to file
suggestion_path = '/content/drive/MyDrive/Colab Notebooks/Stock historical data/Buy suggestion/'
file_name = 'suggestion_' + datetime.date.today().strftime('%d%m%Y') + '.csv'
df.to_csv(suggestion_path + file_name)